### Royale Scrape II
The goal of this notebook is perform card valuations

In [5]:
from bs4 import BeautifulSoup
import requests
from pandas import ExcelWriter, DataFrame
import pandas as pd

import datetime

In [7]:
datetime.datetime.now()

datetime.datetime(2020, 7, 5, 13, 44, 7, 686630)

In [ ]:
def parseURL(link):
    response = requests.get(link).text
    soup = BeautifulSoup(response, 'html.parser')
    return soup

### Trending Cards

In [ ]:
class Card:
    def __init__(self, card_soup):
        # TODO there is some more interesting data in the soup
        self.name = card_soup.find("a")["href"][len('https://statsroyale.com/card/'):]
        self.usage = float(card_soup["data-usage"])
        self.winrate = float(card_soup["data-winrate"])
        
    def __repr__(self):
        return "{}: {}% usage {}% winrate".format(self.name, self.usage, self.winrate)
    
    def __hash__(self):
        return hash(self.name)
    
    def to_row(self):
        return {"Name" : self.name, "Usage": self.usage, "Winrate" : self.winrate}

In [ ]:
def to_df():
    # by default this will sort by usage rate
    most_popular_cards_url = "https://statsroyale.com/top/cards?arena=13"
    soup = parseURL(most_popular_cards_url)

    # all cards are of class: popularCards__card
    all_cards = soup.findAll("div", {"class": "popularCards__card"})
    cards = [Card(tmp) for tmp in all_cards]
    cards = list(set(cards)) # remove dupes
    df = pd.DataFrame(list(map(Card.to_row, cards))); df.index = df.Name; del df["Name"]
    return df

In [ ]:
# note that this changes very quickly
# df = to_df()
df.sort_values('Winrate', ascending=False)